In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'
REMOTE_VIDEOS = "/mnt/rts"
OUTDIR = 'data'



/home/kikohs/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple

In [3]:
# LOCAL imports
import rts

LOG = rts.utils.get_logger()

/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# vidx = rts.ingest.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.ingest.read_all_metadata(METADATA, vidx)
# rts.ingest.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.ingest.load_metadata_hdf5(OUTDIR, 'rts_metadata')

In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
midx.get_stats().number_of_documents

In [ ]:
#TODO get unique geo, people, for the whole media

In [ ]:
a = df.groupby('collection')['collection'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'collections.csv'))

In [ ]:
a = df.groupby('contentType')['contentType'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'contentType.csv'))

In [ ]:
midx.delete_all_documents()

In [ ]:
rts.db.fulltext.index_media(df, midx)

In [34]:
r = rts.io.archive.transcribe_media(os.path.join(OUTDIR, 'myaudio.m4a'))

In [ ]:
df.publishedBy.apply(lambda x: len(x) if x else 0).unique()

In [ ]:
# midx = rts.db.meilisearch.get_or_create_media_index(client)
midx.get_stats().number_of_documents

In [ ]:
%ls {VIDEOS}

In [47]:
video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)

rts.io.archive.create_optimized_media(video_part)


'/mnt/d/rts/0/0/0/ZB001000/export'

In [38]:
a = os.path.join(VIDEOS, '0', '0', 'ZB002000', 'export', 'ZB002000_audio.m4a')
r = rts.io.archive.transcribe_media(a, 'small')

INFO:RTS:Load model: small


In [40]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

True

In [19]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

INFO:RTS:to_mp3 => 59664.23749923706 ms


'data/audio.mp3'

In [34]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

INFO:RTS:to_mp3 => 59204.33855056763 ms


'data/audio.mp3'

In [38]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

INFO:RTS:remux_audio => 343.6279296875 ms


'data/myaudio.m4a'

In [37]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

/mnt/d/rts/0/0/0/ZB001000/ZB001000_track1_dashinit.mp4


In [15]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [30]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
Going to loop over dir data/thumbs
Found total 1349 images to run on
Found total 1349 images to run on■■■■■■■■■■■■■■■■■■] 100% Estimated: 0 Minutes 0 Features Minutes 0 Features
93) Finished write_index() NN model
Stored nn model index file notebook/out/nnf.index
Total time took 2154 ms
Found a total of 653 fully identical images (d>0.990), which are 5.38 %
Found a total of 1089 nearly identical images(d>0.980), which are 8.97 %
Found a total of 5126 above threshold images (d>0.900), which are 42.22 %
Found a total of 539 outlier images         (d<0.050), which are 4.44 %
Min distance found 0.373 max distance 1.000
1671538480 : INFO:     (add_vertices:460): Num vertices for group 0: 1349
1671538480 : INFO:     (commit_edge_buffer:609): In commit edge buffer (0,0)
1671538480 : INFO:     (commit_edge_buffer:680): Shuffling edges ...
1671538480 : INFO:     (commit_edge_buffer:688): Done shuffling edges in 0.005069 

0

In [31]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})

In [32]:
df.len.sum()

986

In [35]:
df

,component_id,files,distance,len
183,451,"[data/thumbs/night-sky.10600448.jpg, data/thumbs/night-sky.10626048.jpg, data/thumbs/night-sky.10651648.jpg, data/thumbs/night-sky.10677248.jpg, data/thumbs/night-sky.10702848.jpg, data/thumbs/night-sky.10728448.jpg, data/thumbs/night-sky.10754048.jpg, data/thumbs/night-sky.10779648.jpg, data/thumbs/night-sky.10805248.jpg, data/thumbs/night-sky.10830848.jpg, data/thumbs/night-sky.10856448.jpg, data/thumbs/night-sky.10882048.jpg, data/thumbs/night-sky.10907648.jpg, data/thumbs/night-sky.10933248.jpg, data/thumbs/night-sky.1221632.jpg, data/thumbs/night-sky.1247232.jpg, data/thumbs/night-sky.1272832.jpg, data/thumbs/night-sky.12982272.jpg, data/thumbs/night-sky.1298432.jpg, data/thumbs/night-sky.13007872.jpg, data/thumbs/night-sky.13033472.jpg, data/thumbs/night-sky.13059072.jpg, data/thumbs/night-sky.1324032.jpg, data/thumbs/night-sky.13353984.jpg, data/thumbs/night-sky.13379584.jpg, data/thumbs/night-sky.13405184.jpg, data/thumbs/night-sky.13430784.jpg, data/thumbs/night-sky.13456384.jpg, data/thumbs/night-sky.13481984.jpg, data/thumbs/night-sky.1349632.jpg, data/thumbs/night-sky.13507584.jpg, data/thumbs/night-sky.13533184.jpg, data/thumbs/night-sky.13558784.jpg, data/thumbs/night-sky.13584384.jpg, data/thumbs/night-sky.13609984.jpg, data/thumbs/night-sky.13635584.jpg, data/thumbs/night-sky.13661184.jpg, data/thumbs/night-sky.1375232.jpg, data/thumbs/night-sky.1400832.jpg, data/thumbs/night-sky.1426432.jpg, data/thumbs/night-sky.15896576.jpg, data/thumbs/night-sky.15922176.jpg, data/thumbs/night-sky.15947776.jpg, data/thumbs/night-sky.15973376.jpg, data/thumbs/night-sky.15998976.jpg, data/thumbs/night-sky.16231424.jpg, data/thumbs/night-sky.16257024.jpg, data/thumbs/night-sky.16282624.jpg, data/thumbs/night-sky.16308224.jpg, data/thumbs/night-sky.16333824.jpg, data/thumbs/night-sky.16359424.jpg, data/thumbs/night-sky.16385024.jpg, data/thumbs/night-sky.16410624.jpg, data/thumbs/night-sky.16436224.jpg, data/thumbs/night-sky.17835520.jpg, data/thumbs/night-sky.17861120.jpg, data/thumbs/night-sky.17886720.jpg, data/thumbs/night-sky.17912320.jpg, data/thumbs/night-sky.17937920.jpg, data/thumbs/night-sky.17963520.jpg, data/thumbs/night-sky.18614784.jpg, data/thumbs/night-sky.18640384.jpg, data/thumbs/night-sky.18665984.jpg, data/thumbs/night-sky.18691584.jpg, data/thumbs/night-sky.18717184.jpg, data/thumbs/night-sky.18742784.jpg, data/thumbs/night-sky.18768384.jpg, data/thumbs/night-sky.18793984.jpg, data/thumbs/night-sky.18819584.jpg, data/thumbs/night-sky.18845184.jpg, data/thumbs/night-sky.18870784.jpg, data/thumbs/night-sky.20907008.jpg, data/thumbs/night-sky.20932608.jpg, data/thumbs/night-sky.20958208.jpg, data/thumbs/night-sky.20983808.jpg, data/thumbs/night-sky.21009408.jpg, data/thumbs/night-sky.21035008.jpg, data/thumbs/night-sky.21060608.jpg, data/thumbs/night-sky.21086208.jpg, data/thumbs/night-sky.21111808.jpg, data/thumbs/night-sky.21137408.jpg, data/thumbs/night-sky.21163008.jpg, data/thumbs/night-sky.4319232.jpg, data/thumbs/night-sky.4344832.jpg, data/thumbs/night-sky.4370432.jpg, data/thumbs/night-sky.4396032.jpg, data/thumbs/night-sky.4421632.jpg, data/thumbs/night-sky.4447232.jpg, data/thumbs/night-sky.5489664.jpg, data/thumbs/night-sky.5515264.jpg, data/thumbs/night-sky.5540864.jpg, data/thumbs/night-sky.5566464.jpg]",0.900418,92
191,471,"[data/thumbs/night-sky.10011648.jpg, data/thumbs/night-sky.10037248.jpg, data/thumbs/night-sky.10062848.jpg, data/thumbs/night-sky.10088448.jpg, data/thumbs/night-sky.10114048.jpg, data/thumbs/night-sky.10139648.jpg, data/thumbs/night-sky.10165248.jpg, data/thumbs/night-sky.10190848.jpg, data/thumbs/night-sky.10216448.jpg, data/thumbs/night-sky.10242048.jpg, data/thumbs/night-sky.10267648.jpg, data/thumbs/night-sky.10293248.jpg, data/thumbs/night-sky.10318848.jpg, data/thumbs/night-sky.10344448.jpg, data/thumbs/night-sky.10370048.jpg, data/thumbs/night-sky.10395648.jpg, data/thumbs/night-sky.10421248.jpg, data/thumbs/night-sky.1044684